The News Dataset has the following attributes , they are:
1. id: unique id for a news article 
2. title: title of the news article
3. author: author of the news article
4. text: text of the article 
5. label: a label that marks whether the news article is real or fake:
   1 => fake news
   0 => real news
   
Here we use the following tools:
. NLTK - Natural Language ToolKit
. Regular Expression
. sklearn => TfidfVectorizer => to convert the text into the feature vectors (numbers)
          => Logistic Regression is used since this problem is a binary classification Problem where the output can be either 
             a fake (1) or real (0) news.

Basically this data is a text data and we need to convert the data into numeric data as a part of the preprocessing.

The data set is extracted from Kaggle:
https://www.kaggle.com/competitions/fake-news/data

(use train.csv from the files present in the above link)


# 1. Import all the necessary libraries:

In [1]:
# Numpy is used for numerical calculations or to deal with the Numpy arrays
# Pandas is used to deal with the dataframes
# Regular Expressions are used to search through the text
# NLTK is used ; corpus => the important context of the text ; stopwords => words that do not add value to the para ; 
# stemming is the process of extracting the root word from the word by removing their suffices and prefixes 
# TfidfVectorizer => used to convert the text into a feature vector

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english')) 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\17347\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Data Preprocessing:

In [3]:
df = pd.read_csv(r"C:\Users\17347\Sapna's Projects\Fake News Prediction\fake-news\train.csv")

In [4]:
print(df.head)

<bound method NDFrame.head of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2        

In [5]:
df.shape

(20800, 5)

In [6]:
# to print the first 5 rows of the dataset
print(df.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


In [7]:
# counting the number of missing values in each column by using isnull()
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
#Replacing the null values with the empty string
df = df.fillna('')
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
#merging the title and the author name ; content data and the label are used to make the predictions
df['content'] = df['author']+' '+df['title']
print(df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [10]:
#seperating the data and the label 
# axis -> 0 ( row ) ; axis -> 1 ( column )
x = df.drop(columns = 'label', axis = 1)
y = df['label']
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

# 2.1 Stemming:  

Stemming is the process of reducing a word to its root word by removing the suffixes and prefixes.

Ex: actor , actress, acting --> act (root word)


In [11]:
porter_stem  = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [porter_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [15]:
df['content'] = df['content'].apply(stemming)
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [16]:
#seperating the data and label 
x = df['content'].values
y = df['label'].values
print(x)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [17]:
y.shape

(20800,)

# 2.2 TfidfVectorizer -> to convert the text to a feature vector(number)


Tf  -> Term Frequency -> Basically counts the number of times a word is repeating in the document ; repetition tells the model that it is the very important word and it assigns a particular numeric value to that word 

Idf -> Inverse Document Frequency -> A word which is repeated several times doesn't have meaning in it; when you conduct a movie review and if a movie review is repeated again and again in the collective positive and negative reviews ; the movie name doesn't add any meaning ; thus can be ignored 
Finds the word which are repeating so many times and it detects that those words are not significant and it reduces its importance value but the term frequency finds the number of significant words and it will give a particular value to it



In [18]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
#all the values in the content column will be converted to feature vectors ( numbers )
x = vectorizer.transform(x)

In [19]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Now, this transformed text data is going to be fed into the ML model in order to make predictions

# 3. Splitting the dataset into training & test data:

In [20]:
#train_test_split : train -> 80% and test -> 20% ; stratify -> if you don't mention this stratify , then the real news and the 
#fake news won't be segregated in equal proportion and when you use stratify = y there will be a similar proportion as it was in
#original dataset 
#random_state -> the random state function controls the shuffling process , it can be between 0 and 42 
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.2, stratify = y , random_state = 2 )

# 4. Training the Logistic Regression Model:

In [21]:
model = LogisticRegression()

In [24]:
#Train the model using model.fit()
model.fit(x_train,y_train)

LogisticRegression()

# 5. Evaluation:

Accuracy Score:

In [27]:
# accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [28]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9865985576923076


In [29]:
# accuracy score on the test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


# 6. Making a Predictive System:

In [30]:
#x-new is the new news for which we're going to make a prediction; x_test[0] -> first news 
x_new = x_test[0]
prediction = model.predict(x_new)
print(prediction)

if prediction[0] == 1:
    print("The news is Fake")
else:
    print("The news is Real")

[1]
The news is Fake


In [32]:
print(y_test[0])

1


Our model has predicted correctly for the first news item that the news is fake 